<a href="https://colab.research.google.com/github/bangse94/hongik_ee/blob/master/mnist_my_handwriting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator


batch_size = 128

img_rows, img_cols = 28, 28

(x_train, y_train), (x_test, y_test) = mnist.load_data()


x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)


x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255


y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest')


model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    validation_data=(x_test, y_test),
                    epochs=12)

score = model.evaluate(x_test, y_test)
print('evaluate accuracy:', score[1])


model.save('mnist_model.h5')

In [0]:
from tensorflow.keras.models import load_model
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

num = 0

#===========================
# image processing
#===========================
#load image
img_color = cv2.imread("/content/numbers2.jpg", cv2.IMREAD_COLOR)
cv2_imshow(img_color)
cv2.waitKey(0)
#convert image color to gray
img_gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
#make binary image from gray
_,img_binary = cv.threshold(img_gray, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)
#opening(morphology)
kernel = np.ones((5,5), np.uint8)
img_binary = cv2.morphologyEx(img_binary, cv2.MORPH_OPEN, kernel)

cv2_imshow(img_binary)
cv2.waitKey(0)

#detect number contour
contours, hierarchy = cv.findContours(img_binary, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

#predict number
for contour in contours:
    num += 1

    x1, y1, width, height = cv2.boundingRect(contour)

    #test하기 위한 공간 확복
    length = max(width, height) + 60
    img_num = np.zeros((length, length, 1), np.uint8)

    #정사각형으로 window를 만들어주자
    x2, y2 = x1-(length-width)//2, y1-(length-height)//2

    #test공간에 binary image input    
    img_num = img_binary[y2:y2+length,x2:x2+length]

    kernel = np.ones((5,5), np.uint8)
    img_num = cv2.morphologyEx(img_num, cv2.MORPH_CLOSE, kernel)

    cv2_imshow(img_num)
    cv2.waitKey(0)

    #load model and test
    model = load_model('mnist_model.h5')

    img_num = cv2.resize(img_num, (28, 28), interpolation=cv2.INTER_AREA)
    img_num = img_num/255.0
    img_input = img_num.reshape(1, 28, 28, 1)
    predictions = model.predict(img_input)

    answer = np.argmax(predictions)
    print("number" , num, "is " +str(answer))

    #color image에 sketch window
    cv2.rectangle(img_color, (x1, y1), (x1+width, y1+height), (255, 255, 0), 2)

    #window 위에 predict 표시
    location = (x1 + int(width *0.5), y1 - 10)
    font = cv2.FONT_HERSHEY_COMPLEX  
    fontScale = 1.2
    cv2.putText(img_color, str(answer), location, font, fontScale, (0,255,0), 2)

cv2_imshow(img_color)
cv2.waitKey(0)
